Importing all libraries and packages that might be used.

In [1]:
import numpy as np
from bs4 import BeautifulSoup
import requests
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
import json
from pandas.io.json import json_normalize 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.84 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.49 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  48.88 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.04 MB/s
vincent-0.4.4- 100% |###################

Adding utilities.csv file to the notebook from assets in order to work on it

In [2]:

import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_63e80d9c265f429caa11be66c2378321 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='f_xCAD3nyjl5kwhQMo0HCCg9rbdUHyxr9i5zJiNXjfGH',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_63e80d9c265f429caa11be66c2378321.get_object(Bucket='capstone-donotdelete-pr-nxgjdzwrrmtirp',Key='uscities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


Filtering the dataset to keep just place and coordinate columns for Washington district in Pennysylvania

In [3]:
df=df_data_1[['city','county_name','state_name','lat','lng']].copy()
df.head()

,city,county_name,state_name,lat,lng
0,Prairie Ridge,Pierce,Washington,47.1443,-122.1408
1,Edison,Skagit,Washington,48.5602,-122.4311
2,Packwood,Lewis,Washington,46.6085,-121.6702
3,Wautauga Beach,Kitsap,Washington,47.5862,-122.5482
4,Harper,Kitsap,Washington,47.5207,-122.5196


In [4]:
df_PA=df[df['state_name']=='Pennsylvania'].reset_index(drop=True)
washington_data=df_PA[df_PA['county_name']=='Washington'].reset_index(drop=True)
washington_data.head()


,city,county_name,state_name,lat,lng
0,Cross Creek,Washington,Pennsylvania,40.3264,-80.4091
1,Thomas,Washington,Pennsylvania,40.2376,-80.0939
2,Twilight,Washington,Pennsylvania,40.1158,-79.8936
3,Canonsburg,Washington,Pennsylvania,40.2642,-80.1868
4,Atlasburg,Washington,Pennsylvania,40.3436,-80.3803


Obtaining geographical coordinates for Washington district to map it using folium.

In [5]:
address = 'Washington, PA'

geolocator = Nominatim(user_agent="PA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Washington are 40.1943798, -80.2541251.


In [6]:
washington_data.shape

(81, 5)

There are 81 cities in total in the Washington county.

In [7]:
# create map of Washington using latitude and longitude values
map_washington = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(washington_data['lat'], washington_data['lng'], washington_data['city']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington)  
    
map_washington


Passing Credentials to obtain foursquare access

In [8]:
CLIENT_ID = 'UKZ4UV54O0YKGO3WRGJ4IKPDDYAS2LRVISRLIP2IZZEX1GJS' # your Foursquare ID
CLIENT_SECRET = '2P1U5S4UBPZ2LNNIKBVQ1VZWVDTWGA5GOKT35HBQJHEJHTPS' # your Foursquare Secret
VERSION = '20190121' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ',CLIENT_ID)
print('CLIENT_SECRET:' , CLIENT_SECRET)

Your credentails:
CLIENT_ID:  UKZ4UV54O0YKGO3WRGJ4IKPDDYAS2LRVISRLIP2IZZEX1GJS
CLIENT_SECRET: 2P1U5S4UBPZ2LNNIKBVQ1VZWVDTWGA5GOKT35HBQJHEJHTPS


Getting all restaurants in the county by adding section=food

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT=100
        section='food'
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            section)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city', 
                  'city Latitude', 
                  'city Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
washington_venues = getNearbyVenues(names=washington_data['city'],
                                   latitudes=washington_data['lat'],
                                   longitudes=washington_data['lng']
                                  )

Cross Creek
Thomas
Twilight
Canonsburg
Atlasburg
Primrose
Centerville
Avella
Hickory
Muse
Beallsville
Washington
Burgettstown
East Washington
McDonald
McGovern
Courtney
Finleyville
Slovan
West Middletown
Cokeburg
North Charleroi
Marianna
Elco
Ellsworth
Paris
Langeloth
Wylandville
Joffre
Wolfdale
Hendersonville
Taylorstown
Southview
Green Hills
Fisher
Bentleyville
Midway
Van Voorhis
Venice
Eighty Four
Dunlevy
Monongahela
Woodrow
Moninger
Roscoe
Murry Hill
Taylorstown Station
Wickerham Manor
Charleroi
Thompsonville
Elrama
Cecil
Manifold
California
Coal Center
Speers
Donora
Strabane
McMurray
New Eagle
Cowden
Gastonville
Meadow Lands
Candor
Millsboro
Westland
Buffalo
Fredericktown
Deemston
Claysville
Bishop
Bulger
Linden
Long Branch
Stockdale
West Alexander
Houston
West Brownsville
Lawrence
Baidland
Allenport


In [11]:
print(washington_venues.shape)
washington_venues.head()


(250, 7)


,city,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Twilight,40.1158,-79.8936,BJ's Diner,40.122878,-79.890087,Diner
1,Canonsburg,40.2642,-80.1868,SUBWAY,40.260099,-80.181431,Sandwich Place
2,Canonsburg,40.2642,-80.1868,Little Caesars Pizza,40.259645,-80.182212,Pizza Place
3,Canonsburg,40.2642,-80.1868,Big Fella's Pizza and Wings,40.259135,-80.185195,Pizza Place
4,Canonsburg,40.2642,-80.1868,Vocelli Pizza,40.258674,-80.187047,Pizza Place


So we can see there are 250 restaurants in the entire county.

In [82]:
food_group=washington_venues.groupby('city').count()
food_group=food_group.drop(['city Latitude','city Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
food_group=food_group.rename(columns = {"Venue":"Food Venue"})
food_group

,Food Venue
city,
Allenport,1
Atlasburg,2
Avella,2
Baidland,2
Beallsville,1
Bentleyville,4
Bishop,4
Burgettstown,9
Canonsburg,13


In [13]:
print('There are {} uniques categories.'.format(len(washington_venues['Venue Category'].unique())))

There are 33 uniques categories.


There are 33 types of restaurants in the county. We create an instance for each restaurant by one hot encoding.

In [14]:
washington_onehot = pd.get_dummies(washington_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
washington_onehot['city'] = washington_venues['city'] 

# move city column to the first column
fixed_columns = [washington_onehot.columns[-1]] + list(washington_onehot.columns[:-1])
washington_onehot = washington_onehot[fixed_columns]

washington_onehot.head()


,city,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,...,New American Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Taco Place,Wings Joint
0,Twilight,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Canonsburg,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Canonsburg,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Canonsburg,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Canonsburg,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
washington_onehot.shape

(250, 34)

In [16]:
washington_grouped = washington_onehot.groupby('city').mean().reset_index()
washington_grouped


,city,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,...,New American Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Taco Place,Wings Joint
0,Allenport,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.000000,1.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
1,Atlasburg,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
2,Avella,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.500000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
3,Baidland,0.500000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
4,Beallsville,0.000000,0.000000,0.0,0.000000,1.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
5,Bentleyville,0.250000,0.250000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.250000,0.00,0.00,0.000000,0.000000,0.00,0.000000
6,Bishop,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.250000,0.000000,0.000000,0.00,0.25,0.000000,0.000000,0.00,0.000000
7,Burgettstown,0.111111,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.111111,...,0.0,0.111111,0.000000,0.111111,0.00,0.00,0.000000,0.000000,0.00,0.000000
8,Canonsburg,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0,0.461538,0.000000,0.153846,0.00,0.00,0.000000,0.000000,0.00,0.000000
9,Charleroi,0.000000,0.000000,0.1,0.000000,0.000000,0.1000,0.000000,0.000000,0.000000,...,0.0,0.100000,0.000000,0.100000,0.00,0.00,0.000000,0.000000,0.00,0.000000


As we see above only 50 cities out of 81 have some kind of a restaurant. 

In [17]:
num_top_venues = 34

for hood in washington_grouped['city']:
    print("----"+hood+"----")
    temp = washington_grouped[washington_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Allenport----
                      venue  freq
0                Restaurant   1.0
1       American Restaurant   0.0
2         German Restaurant   0.0
3                Taco Place   0.0
4          Sushi Restaurant   0.0
5                Steakhouse   0.0
6               Snack Place   0.0
7        Seafood Restaurant   0.0
8            Sandwich Place   0.0
9               Pizza Place   0.0
10  New American Restaurant   0.0
11       Mexican Restaurant   0.0
12        Korean Restaurant   0.0
13       Italian Restaurant   0.0
14            Hot Dog Joint   0.0
15         Greek Restaurant   0.0
16      Fried Chicken Joint   0.0
17         Asian Restaurant   0.0
18               Food Truck   0.0
19                     Food   0.0
20     Fast Food Restaurant   0.0
21               Donut Shop   0.0
22                    Diner   0.0
23            Deli / Bodega   0.0
24  Comfort Food Restaurant   0.0
25       Chinese Restaurant   0.0
26                     Café   0.0
27             Burger Joint   

                      venue  freq
0                      Food  0.22
1       American Restaurant  0.11
2            Sandwich Place  0.11
3      Fast Food Restaurant  0.11
4               Pizza Place  0.11
5                     Diner  0.11
6        Chinese Restaurant  0.11
7   Comfort Food Restaurant  0.11
8        Seafood Restaurant  0.00
9             Hot Dog Joint  0.00
10               Restaurant  0.00
11              Snack Place  0.00
12               Steakhouse  0.00
13         Sushi Restaurant  0.00
14               Taco Place  0.00
15  New American Restaurant  0.00
16       Mexican Restaurant  0.00
17        Korean Restaurant  0.00
18       Italian Restaurant  0.00
19      Fried Chicken Joint  0.00
20         Greek Restaurant  0.00
21        German Restaurant  0.00
22         Asian Restaurant  0.00
23               Food Truck  0.00
24               Donut Shop  0.00
25            Deli / Bodega  0.00
26                     Café  0.00
27             Burger Joint  0.00
28           B

                      venue  freq
0            Sandwich Place   1.0
1       American Restaurant   0.0
2         German Restaurant   0.0
3                Taco Place   0.0
4          Sushi Restaurant   0.0
5                Steakhouse   0.0
6               Snack Place   0.0
7        Seafood Restaurant   0.0
8                Restaurant   0.0
9               Pizza Place   0.0
10  New American Restaurant   0.0
11       Mexican Restaurant   0.0
12        Korean Restaurant   0.0
13       Italian Restaurant   0.0
14            Hot Dog Joint   0.0
15         Greek Restaurant   0.0
16      Fried Chicken Joint   0.0
17         Asian Restaurant   0.0
18               Food Truck   0.0
19                     Food   0.0
20     Fast Food Restaurant   0.0
21               Donut Shop   0.0
22                    Diner   0.0
23            Deli / Bodega   0.0
24  Comfort Food Restaurant   0.0
25       Chinese Restaurant   0.0
26                     Café   0.0
27             Burger Joint   0.0
28           B

                      venue  freq
0        Italian Restaurant  0.67
1       American Restaurant  0.33
2         German Restaurant  0.00
3                Taco Place  0.00
4          Sushi Restaurant  0.00
5                Steakhouse  0.00
6               Snack Place  0.00
7        Seafood Restaurant  0.00
8            Sandwich Place  0.00
9                Restaurant  0.00
10              Pizza Place  0.00
11  New American Restaurant  0.00
12       Mexican Restaurant  0.00
13        Korean Restaurant  0.00
14            Hot Dog Joint  0.00
15         Greek Restaurant  0.00
16      Fried Chicken Joint  0.00
17         Asian Restaurant  0.00
18               Food Truck  0.00
19                     Food  0.00
20     Fast Food Restaurant  0.00
21               Donut Shop  0.00
22                    Diner  0.00
23            Deli / Bodega  0.00
24  Comfort Food Restaurant  0.00
25       Chinese Restaurant  0.00
26                     Café  0.00
27             Burger Joint  0.00
28           B

                      venue  freq
0       American Restaurant  0.33
1                Steakhouse  0.33
2                Restaurant  0.33
3         German Restaurant  0.00
4                Taco Place  0.00
5          Sushi Restaurant  0.00
6               Snack Place  0.00
7        Seafood Restaurant  0.00
8            Sandwich Place  0.00
9               Pizza Place  0.00
10  New American Restaurant  0.00
11       Mexican Restaurant  0.00
12        Korean Restaurant  0.00
13       Italian Restaurant  0.00
14            Hot Dog Joint  0.00
15         Greek Restaurant  0.00
16      Fried Chicken Joint  0.00
17         Asian Restaurant  0.00
18               Food Truck  0.00
19                     Food  0.00
20     Fast Food Restaurant  0.00
21               Donut Shop  0.00
22                    Diner  0.00
23            Deli / Bodega  0.00
24  Comfort Food Restaurant  0.00
25       Chinese Restaurant  0.00
26                     Café  0.00
27             Burger Joint  0.00
28           B

                      venue  freq
0                      Food   1.0
1       American Restaurant   0.0
2               Pizza Place   0.0
3             Hot Dog Joint   0.0
4        Italian Restaurant   0.0
5         Korean Restaurant   0.0
6        Mexican Restaurant   0.0
7   New American Restaurant   0.0
8                Restaurant   0.0
9         German Restaurant   0.0
10           Sandwich Place   0.0
11       Seafood Restaurant   0.0
12              Snack Place   0.0
13               Steakhouse   0.0
14         Sushi Restaurant   0.0
15               Taco Place   0.0
16         Greek Restaurant   0.0
17      Fried Chicken Joint   0.0
18         Asian Restaurant   0.0
19               Food Truck   0.0
20     Fast Food Restaurant   0.0
21               Donut Shop   0.0
22                    Diner   0.0
23            Deli / Bodega   0.0
24  Comfort Food Restaurant   0.0
25       Chinese Restaurant   0.0
26                     Café   0.0
27             Burger Joint   0.0
28           B

                      venue  freq
0       American Restaurant   0.5
1                Restaurant   0.5
2         German Restaurant   0.0
3                Taco Place   0.0
4          Sushi Restaurant   0.0
5                Steakhouse   0.0
6               Snack Place   0.0
7        Seafood Restaurant   0.0
8            Sandwich Place   0.0
9               Pizza Place   0.0
10  New American Restaurant   0.0
11       Mexican Restaurant   0.0
12        Korean Restaurant   0.0
13       Italian Restaurant   0.0
14            Hot Dog Joint   0.0
15         Greek Restaurant   0.0
16      Fried Chicken Joint   0.0
17         Asian Restaurant   0.0
18               Food Truck   0.0
19                     Food   0.0
20     Fast Food Restaurant   0.0
21               Donut Shop   0.0
22                    Diner   0.0
23            Deli / Bodega   0.0
24  Comfort Food Restaurant   0.0
25       Chinese Restaurant   0.0
26                     Café   0.0
27             Burger Joint   0.0
28           B

For cities that have some kind of a restaurant lets find out the least frequent types. These are the types we shall target in our strategy.

In [18]:
def return_least_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 33

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Least Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Least Common Venue'.format(ind+1))

# create a new dataframe
citys_venues_sorted = pd.DataFrame(columns=columns)
citys_venues_sorted['city'] = washington_grouped['city']

for ind in np.arange(washington_grouped.shape[0]):
    citys_venues_sorted.iloc[ind, 1:] = return_least_common_venues(washington_grouped.iloc[ind, :], num_top_venues)

citys_venues_sorted.head()


,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,24th Least Common Venue,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue
0,Allenport,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Pizza Place,New American Restaurant,Mexican Restaurant,...,Burger Joint,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Café,Wings Joint,Restaurant
1,Atlasburg,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Food Truck,Café,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Chinese Restaurant,Wings Joint,Hot Dog Joint
2,Avella,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Food Truck,Wings Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Donut Shop,Fast Food Restaurant,Café,Deli / Bodega,Pizza Place
3,Baidland,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Café,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Diner,American Restaurant
4,Beallsville,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Chinese Restaurant,Café,Burger Joint,Breakfast Spot,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Bakery


We run Kmeans clustering unsupervised algorithm to divide cities into cluster areas to find out areas where maximum restaurants are clustered.

In [22]:
# set number of clusters
kclusters = 8

washington_grouped_clustering = washington_grouped.drop('city', 1)

# run k-means clustering using 350 iterations
kmeans = KMeans(n_clusters=kclusters, n_init=350, random_state=0).fit(washington_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]





array([0, 7, 2, 6, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 1, 4, 1, 1, 2, 4, 0,
       4, 4, 0, 4, 4, 4, 0, 3, 3, 4, 4, 0, 6, 1, 4, 1, 3, 1, 0, 0, 3, 4, 6,
       4, 0, 5, 3], dtype=int32)

In [24]:
washington_merged = washington_data
washington_merged = washington_merged.join(citys_venues_sorted.set_index('city'), on='city')
# Keep only the cities that have at least 1 restaurant i.e 50 out of 81
washington_merged = washington_merged[pd.notnull(washington_merged['1st Least Common Venue'])]
# add clustering labels
washington_merged['Cluster Labels'] = kmeans.labels_

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(washington_merged['lat'], washington_merged['lng'], washington_merged['city'], washington_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
washington_merged.loc[washington_merged['Cluster Labels'] == 0, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
2,Twilight,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Diner,0
41,Monongahela,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Deli / Bodega,Diner,Donut Shop,Chinese Restaurant,Food,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Pizza Place,0
45,Murry Hill,Fried Chicken Joint,Snack Place,Seafood Restaurant,Sandwich Place,Pizza Place,Mexican Restaurant,Korean Restaurant,Hot Dog Joint,Greek Restaurant,...,Bagel Shop,Sushi Restaurant,Steakhouse,New American Restaurant,Fast Food Restaurant,Burger Joint,Italian Restaurant,Restaurant,American Restaurant,0
50,Elrama,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Deli / Bodega,Food,0
57,Strabane,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Wings Joint,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Café,Pizza Place,0
69,Claysville,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Food Truck,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Food,Pizza Place,Café,0
70,Bishop,American Restaurant,Sushi Restaurant,Steakhouse,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,Hot Dog Joint,...,Comfort Food Restaurant,Deli / Bodega,Diner,Café,Wings Joint,Korean Restaurant,Pizza Place,Italian Restaurant,Snack Place,0
78,Lawrence,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Wings Joint,Comfort Food Restaurant,Deli / Bodega,Diner,Fast Food Restaurant,Chinese Restaurant,Donut Shop,Café,Italian Restaurant,0


In [27]:
washington_merged.loc[washington_merged['Cluster Labels'] == 1, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
25,Paris,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Food,1
34,Fisher,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Deli / Bodega,1
35,Bentleyville,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,Pizza Place,New American Restaurant,Mexican Restaurant,...,Food Truck,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food,Sandwich Place,American Restaurant,1
59,New Eagle,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Food Truck,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Pizza Place,Food,1
61,Gastonville,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Bagel Shop,Breakfast Spot,Bakery,Fast Food Restaurant,Pizza Place,Café,Italian Restaurant,Sandwich Place,American Restaurant,1
64,Millsboro,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Wings Joint,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Café,Pizza Place,1


In [28]:
washington_merged.loc[washington_merged['Cluster Labels'] == 2, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
4,Atlasburg,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Café,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Chinese Restaurant,Wings Joint,Hot Dog Joint,2
36,Midway,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Wings Joint,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Café,Pizza Place,2


In [29]:
washington_merged.loc[washington_merged['Cluster Labels'] == 3, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
21,North Charleroi,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Café,Food,Restaurant,Bakery,Pizza Place,3
52,Manifold,Fried Chicken Joint,Pizza Place,Hot Dog Joint,Greek Restaurant,German Restaurant,Taco Place,Restaurant,Food,Seafood Restaurant,...,Italian Restaurant,Food Truck,Asian Restaurant,Mexican Restaurant,Fast Food Restaurant,Burger Joint,American Restaurant,Sandwich Place,Steakhouse,3
54,Coal Center,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Food,Breakfast Spot,BBQ Joint,Bakery,Bagel Shop,Pizza Place,Chinese Restaurant,Sandwich Place,American Restaurant,3
62,Meadow Lands,Fried Chicken Joint,Sushi Restaurant,Snack Place,Seafood Restaurant,Sandwich Place,Pizza Place,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Café,Restaurant,Steakhouse,American Restaurant,3
74,Stockdale,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Café,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Burger Joint,Restaurant,Pizza Place,3
80,Allenport,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Pizza Place,New American Restaurant,Mexican Restaurant,...,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Café,Wings Joint,Restaurant,3


Cluster 4 seems to be the largest so we shall keep it in a dataframe for future use.

In [105]:
food_clustered=washington_merged.loc[washington_merged['Cluster Labels'] == 4, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]
food_clustered=food_clustered.set_index('city', drop=True)
food_clustered

,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,10th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
city,,,,,,,,,,,,,,,,,,,,,
Hickory,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,Mexican Restaurant,...,Burger Joint,Breakfast Spot,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Food,Bakery,American Restaurant,4
Muse,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,Mexican Restaurant,...,Café,Comfort Food Restaurant,Deli / Bodega,Donut Shop,Fast Food Restaurant,Chinese Restaurant,Wings Joint,Italian Restaurant,Diner,4
Beallsville,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,Mexican Restaurant,...,Café,Burger Joint,Breakfast Spot,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Bakery,4
Washington,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,New American Restaurant,Mexican Restaurant,Korean Restaurant,German Restaurant,Food Truck,Food,...,Restaurant,Sandwich Place,Seafood Restaurant,Taco Place,Chinese Restaurant,Hot Dog Joint,Italian Restaurant,American Restaurant,Pizza Place,4
Burgettstown,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,Italian Restaurant,...,Breakfast Spot,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Comfort Food Restaurant,Diner,Sandwich Place,American Restaurant,Food,4
East Washington,Wings Joint,Korean Restaurant,Italian Restaurant,Hot Dog Joint,Greek Restaurant,German Restaurant,Taco Place,Food Truck,Seafood Restaurant,Snack Place,...,Fried Chicken Joint,Mexican Restaurant,Food,Fast Food Restaurant,Diner,Breakfast Spot,Donut Shop,American Restaurant,Pizza Place,4
McDonald,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,Hot Dog Joint,Greek Restaurant,...,BBQ Joint,Bagel Shop,Bakery,Food,Restaurant,Italian Restaurant,Sandwich Place,American Restaurant,Pizza Place,4
Finleyville,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,Italian Restaurant,...,Chinese Restaurant,Comfort Food Restaurant,Deli / Bodega,Diner,Donut Shop,Café,Sandwich Place,Pizza Place,Food,4
Elco,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,Mexican Restaurant,...,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Wings Joint,Food,4


In [31]:
washington_merged.loc[washington_merged['Cluster Labels'] == 5, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]                            
                             

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
7,Avella,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Wings Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Donut Shop,Fast Food Restaurant,Café,Deli / Bodega,Pizza Place,5
79,Baidland,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Diner,American Restaurant,5


In [32]:
washington_merged.loc[washington_merged['Cluster Labels'] == 5, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
7,Avella,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,New American Restaurant,Mexican Restaurant,...,Wings Joint,Chinese Restaurant,Comfort Food Restaurant,Diner,Donut Shop,Fast Food Restaurant,Café,Deli / Bodega,Pizza Place,5
79,Baidland,Fried Chicken Joint,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Sandwich Place,Restaurant,Pizza Place,New American Restaurant,...,Burger Joint,Breakfast Spot,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food Truck,Diner,American Restaurant,5


In [33]:
washington_merged.loc[washington_merged['Cluster Labels'] == 7, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels
3,Canonsburg,American Restaurant,Sushi Restaurant,Steakhouse,Snack Place,Seafood Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Korean Restaurant,...,Deli / Bodega,Donut Shop,Chinese Restaurant,Wings Joint,Greek Restaurant,Diner,Sandwich Place,Food,Pizza Place,7


In [34]:
washington_merged.loc[washington_merged['Cluster Labels'] == 8, washington_merged.columns[[0] + list(range(5, washington_merged.shape[1]))]]

,city,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,...,25th Least Common Venue,26th Least Common Venue,27th Least Common Venue,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels


Then we shall get the shopping areas in Washington county by using section= shops.

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    shops_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT=100
        section='shops'
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            section)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        shops_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_shops = pd.DataFrame([item for shop_list in shops_list for item in shop_list])
    nearby_shops.columns = ['city', 
                  'city Latitude', 
                  'city Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_shops)


In [36]:
washington_shops = getNearbyVenues(names=washington_data['city'],
                                   latitudes=washington_data['lat'],
                                   longitudes=washington_data['lng']
                                  )

Cross Creek
Thomas
Twilight
Canonsburg
Atlasburg
Primrose
Centerville
Avella
Hickory
Muse
Beallsville
Washington
Burgettstown
East Washington
McDonald
McGovern
Courtney
Finleyville
Slovan
West Middletown
Cokeburg
North Charleroi
Marianna
Elco
Ellsworth
Paris
Langeloth
Wylandville
Joffre
Wolfdale
Hendersonville
Taylorstown
Southview
Green Hills
Fisher
Bentleyville
Midway
Van Voorhis
Venice
Eighty Four
Dunlevy
Monongahela
Woodrow
Moninger
Roscoe
Murry Hill
Taylorstown Station
Wickerham Manor
Charleroi
Thompsonville
Elrama
Cecil
Manifold
California
Coal Center
Speers
Donora
Strabane
McMurray
New Eagle
Cowden
Gastonville
Meadow Lands
Candor
Millsboro
Westland
Buffalo
Fredericktown
Deemston
Claysville
Bishop
Bulger
Linden
Long Branch
Stockdale
West Alexander
Houston
West Brownsville
Lawrence
Baidland
Allenport


In [37]:
print(washington_shops.shape)
washington_shops.head()


(487, 7)


,city,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Thomas,40.2376,-80.0939,Pine Ridge Minimart,40.238685,-80.094881,Convenience Store
1,Thomas,40.2376,-80.0939,Re-Bath of Pittsburgh,40.240303,-80.095882,Shop & Service
2,Thomas,40.2376,-80.0939,ABH Supply LLC,40.237849,-80.089806,Business Service
3,Thomas,40.2376,-80.0939,ABH Stair & Millwork Company,40.236371,-80.087908,Paper / Office Supplies Store
4,Canonsburg,40.2642,-80.1868,Ross Beverage,40.259916,-80.181603,Liquor Store


There are 487 shopping centers in Washington county

In [81]:
shop_group=washington_shops.groupby('city').count()
shop_group=shop_group.drop(['city Latitude','city Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
shop_group=shop_group.rename(columns = {"Venue":"Shop Venue"})
shop_group #This creates a dataframe showing number of shopping venues in each city

,Shop Venue
city,
Atlasburg,3
Avella,2
Beallsville,1
Bentleyville,6
Bishop,4
Buffalo,1
Bulger,2
Burgettstown,10
Canonsburg,32


In [39]:
shops_onehot = pd.get_dummies(washington_shops[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
shops_onehot['city'] = washington_shops['city'] 

# move city column to the first column
fixed_columns = [shops_onehot.columns[-1]] + list(shops_onehot.columns[:-1])
shops_onehot = shops_onehot[fixed_columns]

shops_onehot.head()

,city,ATM,Antique Shop,Arts & Crafts Store,Auto Garage,Auto Workshop,Beer Store,Big Box Store,Bookstore,Boutique,...,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Supermarket,Thrift / Vintage Store,Video Store,Warehouse Store
0,Thomas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Thomas,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Thomas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Thomas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Canonsburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Inspection of dataframe shows that there are 76 types of shopping centers. Most cities contain about 10 or less types of shops. So instead of dealing with bulky dataframe with lots if negative values for most shops we shall just filter it to keep the 10 most frequent types of shopping places in each city.

In [40]:
shops_grouped = shops_onehot.groupby('city').mean().reset_index()
shops_grouped.head()

,city,ATM,Antique Shop,Arts & Crafts Store,Auto Garage,Auto Workshop,Beer Store,Big Box Store,Bookstore,Boutique,...,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Supermarket,Thrift / Vintage Store,Video Store,Warehouse Store
0,Atlasburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Avella,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,Beallsville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Bentleyville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0
4,Bishop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [41]:
num_top_venues = 10

for hood in shops_grouped['city']:
    print("----"+hood+"----")
    temp = shops_grouped[shops_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Atlasburg----
                  venue  freq
0     Food & Drink Shop  0.33
1            Shoe Store  0.33
2        Farmers Market  0.33
3           Music Store  0.00
4       Motorcycle Shop  0.00
5     Mobile Phone Shop  0.00
6        Massage Studio  0.00
7                Market  0.00
8  Marijuana Dispensary  0.00
9                   ATM  0.00


----Avella----
                           venue  freq
0               Business Service   0.5
1     Construction & Landscaping   0.5
2  Paper / Office Supplies Store   0.0
3              Other Repair Shop   0.0
4                   Optical Shop   0.0
5                    Music Store   0.0
6                Motorcycle Shop   0.0
7              Mobile Phone Shop   0.0
8                 Massage Studio   0.0
9                         Market   0.0


----Beallsville----
                  venue  freq
0          Home Service   1.0
1  Marijuana Dispensary   0.0
2     Other Repair Shop   0.0
3          Optical Shop   0.0
4           Music Store   0.0
5   

                    venue  freq
0        Business Service   0.2
1             Video Store   0.2
2             Supermarket   0.2
3                Pharmacy   0.2
4  Furniture / Home Store   0.2
5               Locksmith   0.0
6             Music Store   0.0
7         Motorcycle Shop   0.0
8       Mobile Phone Shop   0.0
9          Massage Studio   0.0


----Fredericktown----
                  venue  freq
0            Shoe Store  0.33
1     Convenience Store  0.33
2          Liquor Store  0.33
3  Marijuana Dispensary  0.00
4          Optical Shop  0.00
5           Music Store  0.00
6       Motorcycle Shop  0.00
7     Mobile Phone Shop  0.00
8        Massage Studio  0.00
9                Market  0.00


----Gastonville----
               venue  freq
0   Business Service  0.25
1     Discount Store  0.25
2  Other Repair Shop  0.25
3        Flower Shop  0.25
4       Optical Shop  0.00
5        Music Store  0.00
6    Motorcycle Shop  0.00
7  Mobile Phone Shop  0.00
8     Massage Studio  0.00
9 

                        venue  freq
0  Construction & Landscaping   0.5
1                   Gift Shop   0.5
2                         ATM   0.0
3        Marijuana Dispensary   0.0
4                Optical Shop   0.0
5                 Music Store   0.0
6             Motorcycle Shop   0.0
7           Mobile Phone Shop   0.0
8              Massage Studio   0.0
9                      Market   0.0


----Stockdale----
                  venue  freq
0        Clothing Store   1.0
1                   ATM   0.0
2  Marijuana Dispensary   0.0
3          Optical Shop   0.0
4           Music Store   0.0
5       Motorcycle Shop   0.0
6     Mobile Phone Shop   0.0
7        Massage Studio   0.0
8                Market   0.0
9             Locksmith   0.0


----Strabane----
                        venue  freq
0  Construction & Landscaping  0.26
1               Auto Workshop  0.11
2                Home Service  0.11
3              Discount Store  0.05
4                Carpet Store  0.05
5             Motor

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
citys_shops_sorted = pd.DataFrame(columns=columns)
citys_shops_sorted['city'] = shops_grouped['city']

for ind in np.arange(shops_grouped.shape[0]):
    citys_shops_sorted.iloc[ind, 1:] = return_most_common_venues(shops_grouped.iloc[ind, :], num_top_venues)

citys_shops_sorted.head()

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atlasburg,Shoe Store,Food & Drink Shop,Farmers Market,Flea Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop
1,Avella,Construction & Landscaping,Business Service,Warehouse Store,Farmers Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market
2,Beallsville,Home Service,Warehouse Store,Flea Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market
3,Bentleyville,Other Repair Shop,Video Store,Home Service,Discount Store,IT Services,Pharmacy,Warehouse Store,Fabric Shop,Drugstore,Dry Cleaner
4,Bishop,Construction & Landscaping,Business Service,Warehouse Store,Farmers Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market


We shall divide the county into clusters of areas with similar shopping centers.

In [44]:
# set number of clusters
kclusters = 8

shops_grouped_clustering = shops_grouped.drop('city', 1)

# run k-means clustering with 350 iterations
kmeans = KMeans(n_clusters=kclusters, n_init=350, random_state=0).fit(shops_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 7, 0, 3, 3, 2, 0, 0, 7], dtype=int32)

In [45]:
shops_merged = washington_data
shops_merged = shops_merged.join(citys_shops_sorted.set_index('city'), on='city')
shops_merged = shops_merged[pd.notnull(shops_merged['1st Most Common Venue'])]
# add clustering labels
shops_merged['Cluster Labels'] = kmeans.labels_

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(shops_merged['lat'], shops_merged['lng'], shops_merged['city'], shops_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [106]:
shops_clustered=shops_merged.loc[shops_merged['Cluster Labels'] == 0, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]
shops_clustered=shops_clustered.set_index('city',drop=True)
shops_clustered #Since cluster 1 seems to have maximum cities with shopping areas we shall make a dataframe from it

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
city,,,,,,,,,,,
Thomas,Convenience Store,Paper / Office Supplies Store,Shop & Service,Business Service,Fabric Shop,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,0
Avella,Construction & Landscaping,Business Service,Warehouse Store,Farmers Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market,0
Burgettstown,Business Service,Pharmacy,Grocery Store,Optical Shop,Discount Store,Clothing Store,Pet Store,Event Service,Drugstore,Dry Cleaner,0
East Washington,Lawyer,IT Services,Business Service,Rental Service,Costume Shop,Discount Store,Convenience Store,Laundromat,Photography Studio,Print Shop,0
McGovern,Health & Beauty Service,Construction & Landscaping,Business Service,Farmers Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market,0
Finleyville,Construction & Landscaping,ATM,Flower Shop,Clothing Store,Pharmacy,Check Cashing Service,Liquor Store,Business Service,Grocery Store,Video Store,0
Slovan,Other Repair Shop,Farmers Market,Department Store,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market,0
Cokeburg,Electronics Store,Event Service,Warehouse Store,Costume Shop,Discount Store,Drugstore,Dry Cleaner,Fabric Shop,Farmers Market,Flea Market,0
North Charleroi,Other Repair Shop,Convenience Store,Construction & Landscaping,Business Service,Department Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,0


In [48]:
shops_merged.loc[shops_merged['Cluster Labels'] == 1, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
60,Cowden,Business Service,Home Service,Convenience Store,Insurance Office,Warehouse Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,1


In [49]:
shops_merged.loc[shops_merged['Cluster Labels'] == 2, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
11,Washington,Lawyer,Home Service,Business Service,Construction & Landscaping,Pharmacy,Liquor Store,Convenience Store,IT Services,Department Store,Discount Store,2
30,Hendersonville,Sporting Goods Shop,Furniture / Home Store,Pet Store,Shop & Service,Boutique,Warehouse Store,Fabric Shop,Discount Store,Drugstore,Dry Cleaner,2
44,Roscoe,Discount Store,Convenience Store,Clothing Store,Business Service,Department Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,2
51,Cecil,Home Service,Construction & Landscaping,Warehouse Store,Flea Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,2


In [50]:
shops_merged.loc[shops_merged['Cluster Labels'] == 3, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,Canonsburg,Construction & Landscaping,Business Service,Furniture / Home Store,Flower Shop,Auto Garage,Liquor Store,Mobile Phone Shop,Home Service,Dry Cleaner,Lawyer,3
8,Hickory,Business Service,Home Service,Insurance Office,Child Care Service,Warehouse Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,3
10,Beallsville,Home Service,Warehouse Store,Flea Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,3
31,Taylorstown,Rental Service,Warehouse Store,Farmers Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market,3
34,Fisher,Business Service,Video Store,Supermarket,Furniture / Home Store,Pharmacy,Warehouse Store,Discount Store,Drugstore,Dry Cleaner,Electronics Store,3
45,Murry Hill,Lawyer,Construction & Landscaping,Event Service,Business Service,ATM,Insurance Office,Dry Cleaner,Rental Service,Shopping Mall,Home Service,3
61,Gastonville,Other Repair Shop,Discount Store,Flower Shop,Business Service,Farmers Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,3
75,West Alexander,Lighting Store,Construction & Landscaping,Flea Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,Warehouse Store,3
78,Lawrence,Construction & Landscaping,Arts & Crafts Store,Warehouse Store,Flea Market,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,3


In [51]:
shops_merged.loc[shops_merged['Cluster Labels'] == 4, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
62,Meadow Lands,Business Service,Home Service,Gift Shop,Construction & Landscaping,Warehouse Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,4


In [52]:
shops_merged.loc[shops_merged['Cluster Labels'] == 5, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
66,Buffalo,Construction & Landscaping,Warehouse Store,Farmers Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market,5
67,Fredericktown,Liquor Store,Convenience Store,Shoe Store,Warehouse Store,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,5


In [56]:
shops_merged.loc[shops_merged['Cluster Labels'] == 6, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]


,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
52,Manifold,Sporting Goods Shop,Grocery Store,Furniture / Home Store,Discount Store,Hobby Shop,Print Shop,Fabric Shop,Dry Cleaner,Department Store,Health & Beauty Service,6
71,Bulger,Business Service,Warehouse Store,Farmers Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Flea Market,6


In [54]:
shops_merged.loc[shops_merged['Cluster Labels'] == 7, shops_merged.columns[[0] + list(range(5, shops_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
4,Atlasburg,Shoe Store,Food & Drink Shop,Farmers Market,Flea Market,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,7
14,McDonald,Pharmacy,Discount Store,Business Service,Supermarket,Sporting Goods Shop,Health Food Store,Video Store,Construction & Landscaping,Insurance Office,Mobile Phone Shop,7


We shall find out the cities that have both high number of shopping areas as well as high number of restaurants. We shall also include in the dataframe the absolute number of shops and restarants in the city. This we shall acheive by intersection of the most frequent cluster dataframes and then adding on the absolute number of shops/restaurants dataframes. We shall sum the absolute number of shops and restaurants to find out the absolute number of "Hip Places" in the cities of intersection

In [127]:
washington_hip= pd.concat ([shops_clustered, food_clustered,shop_group,food_group],axis=1)
washington_hip= washington_hip[pd.notnull(washington_hip['1st Least Common Venue'])]
washington_hip= washington_hip[pd.notnull(washington_hip['1st Most Common Venue'])]
washington_hip= washington_hip[pd.notnull(washington_hip['Shop Venue'])]
washington_hip= washington_hip[pd.notnull(washington_hip['Food Venue'])]
washington_hip['Hip Places'] = washington_hip[['Shop Venue', 'Food Venue']].sum(axis=1)
washington_hip.head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,...,28th Least Common Venue,29th Least Common Venue,30th Least Common Venue,31th Least Common Venue,32th Least Common Venue,33th Least Common Venue,Cluster Labels,Shop Venue,Food Venue,Hip Places
Burgettstown,Business Service,Pharmacy,Grocery Store,Optical Shop,Discount Store,Clothing Store,Pet Store,Event Service,Drugstore,Dry Cleaner,...,Chinese Restaurant,Comfort Food Restaurant,Diner,Sandwich Place,American Restaurant,Food,4.0,10.0,9.0,19.0
Charleroi,Business Service,Construction & Landscaping,Home Service,Recording Studio,Furniture / Home Store,Pharmacy,Discount Store,Mobile Phone Shop,Thrift / Vintage Store,Lawyer,...,Pizza Place,Breakfast Spot,Sandwich Place,BBQ Joint,Food,Italian Restaurant,4.0,23.0,10.0,33.0
Donora,Business Service,Discount Store,Grocery Store,Convenience Store,Auto Workshop,Pharmacy,Farmers Market,Drugstore,Dry Cleaner,Electronics Store,...,Bakery,Bagel Shop,BBQ Joint,Asian Restaurant,Food,Pizza Place,4.0,7.0,5.0,12.0
East Washington,Lawyer,IT Services,Business Service,Rental Service,Costume Shop,Discount Store,Convenience Store,Laundromat,Photography Studio,Print Shop,...,Fast Food Restaurant,Diner,Breakfast Spot,Donut Shop,American Restaurant,Pizza Place,4.0,24.0,16.0,40.0
Eighty Four,Business Service,Home Service,Lawyer,Warehouse Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,...,Diner,Donut Shop,Fast Food Restaurant,Café,Wings Joint,Sandwich Place,4.0,4.0,1.0,5.0


Now we shall arrange the 13 cities in these common clusters in order of maximum number of hip places.

In [133]:
washington_hip1=washington_hip[['1st Least Common Venue','2nd Least Common Venue', '3rd Least Common Venue','Hip Places']].copy()
washington_hip2=washington_hip1.sort_values(['Hip Places'], ascending=[False])
washington_hip2

,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,Hip Places
Thompsonville,Fried Chicken Joint,Sushi Restaurant,Steakhouse,55.0
East Washington,Wings Joint,Korean Restaurant,Italian Restaurant,40.0
Charleroi,American Restaurant,Sushi Restaurant,Steakhouse,33.0
Burgettstown,Fried Chicken Joint,Sushi Restaurant,Steakhouse,19.0
Moninger,Fried Chicken Joint,Sushi Restaurant,Steakhouse,16.0
Finleyville,American Restaurant,Sushi Restaurant,Steakhouse,15.0
Donora,American Restaurant,Sushi Restaurant,Steakhouse,12.0
West Brownsville,American Restaurant,Sushi Restaurant,Steakhouse,8.0
Wickerham Manor,American Restaurant,Sushi Restaurant,Steakhouse,7.0
Speers,Fried Chicken Joint,Sushi Restaurant,Steakhouse,6.0


Thus we can see that Thompsonville is the busiest place with maximum shopping and eating places but lacks 'Fried Chicken Joint'. So our recommendation would be a Fried Chicken Joint in Thompsonville.

                             ------------------------XXXXXX------------------------